In [1]:
/* 0) Base: last 2 years of reseller sales */
WITH base AS (
    SELECT
        f.SalesOrderNumber,
        f.SalesAmount,
        f.TotalProductCost,
        f.OrderQuantity,
        f.OrderDate,
        f.ResellerKey,
        f.ProductKey,
        t.SalesTerritoryKey,
        t.SalesTerritoryRegion,
        t.SalesTerritoryCountry,
        t.SalesTerritoryGroup,
        d.CalendarYear,
        d.CalendarQuarter
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    JOIN DimDate d ON f.OrderDateKey = d.DateKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
)

/* 1) Core Answer: Best Territories by Total Sales */
SELECT
    SalesTerritoryRegion,
    SalesTerritoryCountry,
    SalesTerritoryGroup,
    SUM(SalesAmount) AS TotalSales,
    DENSE_RANK() OVER (ORDER BY SUM(SalesAmount) DESC) AS SalesRank
FROM base
GROUP BY SalesTerritoryRegion, SalesTerritoryCountry, SalesTerritoryGroup
ORDER BY SalesRank;


(10 rows affected)

Total execution time: 00:00:00.124

SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup,TotalSales,SalesRank
Southwest,United States,North America,14842999.4326,1
Canada,Canada,North America,11476100.3094,2
Northwest,United States,North America,9325750.9174,3
Central,United States,North America,6056064.8619,4
Northeast,United States,North America,5871590.1975,5
Southeast,United States,North America,5126583.0901,6
France,France,Europe,4607537.935,7
United Kingdom,United Kingdom,Europe,4279008.8266,8
Germany,Germany,Europe,1983988.0373,9
Australia,Australia,Pacific,1594335.3767,10


In [2]:
/* 2) Profit Margin by Territory (correct aggregation) */
WITH base AS (
    SELECT
        f.SalesAmount,
        f.TotalProductCost,
        f.OrderDate,
        t.SalesTerritoryRegion
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
)
SELECT
    SalesTerritoryRegion,
    ROUND(
        (SUM(SalesAmount) - SUM(TotalProductCost)) * 100.0 / NULLIF(SUM(SalesAmount),0)
    ,2) AS ProfitMarginPercent,
    DENSE_RANK() OVER (
        ORDER BY (SUM(SalesAmount) - SUM(TotalProductCost)) * 1.0 / NULLIF(SUM(SalesAmount),0) DESC
    ) AS ProfitRank
FROM base
GROUP BY SalesTerritoryRegion
ORDER BY ProfitRank;


(10 rows affected)

Total execution time: 00:00:00.103

SalesTerritoryRegion,ProfitMarginPercent,ProfitRank
Northeast,1.220000000000000,1
Central,1.010000000000000,2
Canada,0.770000000000000,3
Northwest,0.610000000000000,4
United Kingdom,0.150000000000000,5
Southeast,-0.180000000000000,6
France,-0.810000000000000,7
Southwest,-0.970000000000000,8
Germany,-5.610000000000000,9
Australia,-6.820000000000000,10


In [4]:
/* 3) AOV by Territory (true order-level AOV) */
WITH base AS (
    SELECT
        f.SalesOrderNumber,
        f.SalesAmount,
        t.SalesTerritoryRegion,
        f.OrderDate
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
),
order_totals AS (
    SELECT
        SalesTerritoryRegion,
        SalesOrderNumber,
        SUM(SalesAmount) AS OrderRevenue
    FROM base
    GROUP BY SalesTerritoryRegion, SalesOrderNumber
)
SELECT
    SalesTerritoryRegion,
    ROUND(AVG(OrderRevenue),2) AS AvgOrderValue,
    DENSE_RANK() OVER (ORDER BY AVG(OrderRevenue) DESC) AS AOVRank
FROM order_totals
GROUP BY SalesTerritoryRegion
ORDER BY AOVRank;


(10 rows affected)

Total execution time: 00:00:00.096

SalesTerritoryRegion,AvgOrderValue,AOVRank
Southwest,24862.65,1
France,24508.18,2
United Kingdom,22760.69,3
Northeast,21827.47,4
Northwest,21637.47,5
Canada,21057.06,6
Central,20459.68,7
Germany,15379.75,8
Southeast,14360.18,9
Australia,12754.68,10


In [5]:
/* 4) Active Reseller Depth (count + avg active span) */
WITH base AS (
    SELECT
        f.ResellerKey,
        t.SalesTerritoryRegion,
        f.OrderDate
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
),
reseller_activity AS (
    SELECT
        SalesTerritoryRegion,
        ResellerKey,
        MIN(OrderDate) AS FirstOrderDate,
        MAX(OrderDate) AS LastOrderDate,
        DATEDIFF(MONTH, MIN(OrderDate), MAX(OrderDate)) AS ActiveMonths
    FROM base
    GROUP BY SalesTerritoryRegion, ResellerKey
)
SELECT
    SalesTerritoryRegion,
    COUNT(*) AS ActiveResellers,
    ROUND(AVG(CAST(ActiveMonths AS FLOAT)),2) AS AvgActiveMonths
FROM reseller_activity
GROUP BY SalesTerritoryRegion
ORDER BY ActiveResellers DESC;


(10 rows affected)

Total execution time: 00:00:00.155

SalesTerritoryRegion,ActiveResellers,AvgActiveMonths
Southwest,109,14.23
Canada,98,14.72
Northwest,80,13.95
Southeast,74,13.09
Central,56,13.98
Northeast,45,16.07
United Kingdom,38,13.5
Australia,34,8.38
France,34,14.74
Germany,32,7.59


In [6]:
/* 5) Top Products by Territory (revenue-based) */
WITH base AS (
    SELECT
        f.SalesAmount,
        t.SalesTerritoryRegion,
        p.EnglishProductName,
        f.OrderDate
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    JOIN DimProduct p ON f.ProductKey = p.ProductKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
)
SELECT
    SalesTerritoryRegion,
    EnglishProductName,
    SUM(SalesAmount) AS Revenue,
    DENSE_RANK() OVER (
        PARTITION BY SalesTerritoryRegion
        ORDER BY SUM(SalesAmount) DESC
    ) AS ProductRank
FROM base
GROUP BY SalesTerritoryRegion, EnglishProductName
ORDER BY SalesTerritoryRegion, ProductRank;


(2172 rows affected)

Total execution time: 00:00:00.178

SalesTerritoryRegion,EnglishProductName,Revenue,ProductRank
Australia,"Touring-1000 Yellow, 60",139229.688,1
Australia,"Touring-1000 Blue, 60",137322.432,2
Australia,"Touring-1000 Blue, 46",126329.962,3
Australia,"Touring-1000 Yellow, 46",117486.9696,4
Australia,"Touring-1000 Blue, 50",72952.542,5
Australia,"Touring-1000 Yellow, 50",66563.2344,6
Australia,"Touring-1000 Blue, 54",65800.332,7
Australia,"Touring-2000 Blue, 54",64872.99,8
Australia,"Touring-2000 Blue, 60",59041.71,9
Australia,"Touring-1000 Yellow, 54",48539.6652,10


In [7]:
/* 6) Repeat Purchase Rate by Territory (within 2-year window) */
WITH base AS (
    SELECT
        f.ResellerKey,
        f.SalesOrderNumber,
        t.SalesTerritoryRegion,
        f.OrderDate
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
),
orders AS (
    SELECT
        SalesTerritoryRegion,
        ResellerKey,
        COUNT(DISTINCT SalesOrderNumber) AS OrderCount
    FROM base
    GROUP BY SalesTerritoryRegion, ResellerKey
)
SELECT
    SalesTerritoryRegion,
    ROUND(
        SUM(CASE WHEN OrderCount > 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)
    ,2) AS RepeatPurchaseRatePercent
FROM orders
GROUP BY SalesTerritoryRegion
ORDER BY RepeatPurchaseRatePercent DESC;


(10 rows affected)

Total execution time: 00:00:00.121

SalesTerritoryRegion,RepeatPurchaseRatePercent
France,100.000000000000
Northeast,97.780000000000
United Kingdom,97.370000000000
Canada,94.900000000000
Australia,94.120000000000
Northwest,91.250000000000
Central,91.070000000000
Germany,90.630000000000
Southwest,89.910000000000
Southeast,83.780000000000


In [8]:
/* 7) YoY Retention Rate (aligned to actual last 2 years, not hardcoded) */
WITH base AS (
    SELECT
        f.ResellerKey,
        t.SalesTerritoryRegion,
        d.CalendarYear,
        f.OrderDate
    FROM FactResellerSales f
    JOIN DimSalesTerritory t ON f.SalesTerritoryKey = t.SalesTerritoryKey
    JOIN DimDate d ON f.OrderDateKey = d.DateKey
    WHERE f.OrderDate >= (
        SELECT DATEADD(YEAR, -2, MAX(CAST(OrderDate AS DATE))) 
        FROM FactResellerSales
    )
),
years AS (
    SELECT MIN(CalendarYear) AS Year1, MAX(CalendarYear) AS Year2 FROM base
),
y1 AS (
    SELECT DISTINCT b.SalesTerritoryRegion, b.ResellerKey
    FROM base b CROSS JOIN years y
    WHERE b.CalendarYear = y.Year1
),
y2 AS (
    SELECT DISTINCT b.SalesTerritoryRegion, b.ResellerKey
    FROM base b CROSS JOIN years y
    WHERE b.CalendarYear = y.Year2
)
SELECT
    y1.SalesTerritoryRegion,
    COUNT(y1.ResellerKey) AS Year1_Customers,
    COUNT(CASE WHEN y2.ResellerKey IS NOT NULL THEN 1 END) AS RetainedCustomers,
    ROUND(
        COUNT(CASE WHEN y2.ResellerKey IS NOT NULL THEN 1 END) * 100.0
        / NULLIF(COUNT(y1.ResellerKey),0)
    ,2) AS RetentionRatePercent
FROM y1
LEFT JOIN y2
    ON y1.SalesTerritoryRegion = y2.SalesTerritoryRegion
   AND y1.ResellerKey = y2.ResellerKey
GROUP BY y1.SalesTerritoryRegion
ORDER BY RetentionRatePercent DESC;


Warning: Null value is eliminated by an aggregate or other SET operation.

(8 rows affected)

Total execution time: 00:00:00.280

SalesTerritoryRegion,Year1_Customers,RetainedCustomers,RetentionRatePercent
Canada,21,17,80.950000000000
France,5,4,80.000000000000
Northeast,14,11,78.570000000000
Northwest,21,15,71.430000000000
United Kingdom,3,2,66.670000000000
Southeast,27,16,59.260000000000
Southwest,33,19,57.580000000000
Central,16,9,56.250000000000
